# Data Preparation

In [ ]:
import pandas as pd

## Testing data
- Original Competition Data

In [ ]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

## Training data

In [ ]:
from google.colab import drive
import random

### Loading Data

In [ ]:
drive.mount('/content/drive')

In [ ]:
# All Data

data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ALL_DATA.csv"

train = pd.read_csv(data_path)

In [ ]:
print(len(train))
train.head()

In [ ]:
print(train['dialect'].unique())
print(train['dialect'].value_counts())

In [ ]:
train.loc[train['dialect']=='0', 'dialect'] = 'eg'
print(train['dialect'].value_counts())



---



In [ ]:
# Data augmented with Arabic names and locations

names_loc_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/names_locations_augmented_data.csv")

print(len(names_loc_data))
names_loc_data.head()

In [ ]:
train = pd.concat([train, names_loc_data], ignore_index=True)
print(len(train))
train['dialect'].value_counts()

In [ ]:
# Shuffle the data

train = train.sample(frac=1).reset_index(drop=True)

### Cleaning and preprocessing

In [ ]:
import random
import re
from IPython.display import display, HTML
import re
import ast

In [ ]:
idx = random.randint(0, len(train))
idx

In [ ]:
print(train.tokens[idx])
print(type(train.tokens[idx]))
print(train.tags[idx])
print(type(train.tags[idx]))
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

In [ ]:
tokens_list = ast.literal_eval(train.tokens[idx])
print(tokens_list)
print(type(tokens_list), len(tokens_list))

tags_list = ast.literal_eval(train.tags[idx])
print(tags_list)
print(type(tags_list), len(tags_list))

In [ ]:
# Convert tokens and tags columns to lists

train['tokens'] = train["tokens"].apply(ast.literal_eval)
train['tags'] = train['tags'].apply(ast.literal_eval)

In [ ]:
print(train.tokens[idx])
print(type(train.tokens[idx]))
print(train.tags[idx])
print(type(train.tags[idx]))
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

In [ ]:
# Checking if there is mismatching lengths between tokens and tags

length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")

In [ ]:
# Initialize a list to store discrepancy information
discrepancy_data = []

# Creating a new masked sentence using the correct masker function
train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask, discrepancy_data), axis=1)

# Convert the list of discrepancy data into a DataFrame
discrepancy_df = pd.DataFrame(discrepancy_data)

In [ ]:
tags_to_mask = ['PHONEIMEI',
 'VEHICLEVRM',
 'LITECOINADDRESS',
 'CREDITCARDNUMBER',
 'DATE',
 'NEARBYGPSCOORDINATE',
 'BITCOINADDRESS',
 'GENDER',
 'PERSONNAME',
 'JOBTITLE',
 'TIME',
 'CURRENCY',
 'BIC',
 'MASKEDNUMBER',
 'STREET',
 'MAC',
 'DOB',
 'SECONDARYADDRESS',
 'CREDITCARDISSUER',
 'ZIPCODE',
 'USERAGENT',
 'CURRENCYSYMBOL',
 'JOBTYPE',
 'BUILDINGNUMBER',
 'AGE',
 'MIDDLENAME',
 'CREDITCARDINUMBER',
 'ACCOUNTNUMBER',
 'PIN',
 'FIRSTNAME',
 'ORDINALDIRECTION',
 'PASSWORD',
 'PHONENUMBER',
 'IPV4',
 'CREDITCARDCVV',
 'USERNAME',
 'HEIGHT',
 'CURRENCYCODE',
 'ACCOUNTNAME',
 'IBAN',
 'AMOUNT',
 'PREFIX',
 'VEHICLEVIN',
 'SEX',
 'EMAIL',
 'ETHEREUMADDRESS',
 'IPV6',
 'SSN',
 'URL',
 'LASTNAME',
 'CURRENCYNAME',
 'IP']

In [ ]:
def masker(row, tags_to_mask, discrepancy_list):
    """
    Constructs a masked sentence and records discrepancies, attempting to preserve original formatting.

    Args:
    row: A pandas DataFrame row with 'tokens', 'tags', and 'clean_source' columns.
    tags_to_mask: List of tag entity types to mask.
    discrepancy_list: A list to append discrepancy details.

    Returns:
    The constructed sentence with masked tokens for the given row, attempting to preserve spacing.
    """
    tokens = row['tokens']
    tags = row['tags']
    clean_source = row['clean_source']
    masked_sentence = ""
    current_position = 0

    for i in range(len(tokens)):
        token = tokens[i]
        tag = tags[i]

        # Find the position of the current token in the original string starting from the last processed position
        start_index = clean_source.find(token, current_position)

        if start_index != -1:
            # Append the text from the current position up to the start of the token
            masked_sentence += clean_source[current_position:start_index]

            if tag[2:] in tags_to_mask:
                masked_sentence += '[MASK]'
            else:
                masked_sentence += token

            # Update the current position to the end of the current token
            current_position = start_index + len(token)
        else:
            # If token not found at or after current_position, it indicates a discrepancy
            print(f"Warning: Token '{token}' not found in clean_source at or after position {current_position}\n")

            # Record the discrepancy
            discrepancy_list.append({
                'clean_source': clean_source,
                'tokens': tokens,
                'tags': tags,
                'discrepancy_token': token,
                'discrepancy_position': current_position
            })

            # You might choose how to handle the token that wasn't found -
            # here, we'll just append it unmasked to keep the process going
            masked_sentence += token
            # We don't update current_position based on this token as it wasn't found correctly

    # Append any remaining text after the last token
    masked_sentence += clean_source[current_position:]
    return masked_sentence

In [ ]:
# Initialize a list to store discrepancy information
discrepancy_data = []

# Creating a new masked sentence
train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask, discrepancy_data), axis=1)

# Convert the list of discrepancy data into a DataFrame
discrepancy_df = pd.DataFrame(discrepancy_data)

In [ ]:
print(f"{len(discrepancy_df)} discrepancies are found")

In [ ]:
print(f"{len(discrepancy_df.clean_source.unique())} sentences are detected as discrepancies.")

In [ ]:
# Drop discrepancies

print(f"Original number of rows: {len(train)}")

train = train[~train['clean_source'].isin(discrepancy_df['clean_source'])]

print(f"Number of rows after dropping discrepancies: {len(train)}")

In [ ]:
train[['clean_source','masked_source']]

In [ ]:
train_final = train[['clean_source', 'masked_source']].copy()
train_final.rename(columns={'clean_source': 'source', 'masked_source': 'target'}, inplace = True)

# train_final.head()

# # Save the train data after the updates
# train_final.to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv")

In [ ]:
def merge_consecutive_masks(text):
    """Merges consecutive occurrences of '[MASK]' in a string, preserving a trailing whitespace if present."""
    return re.sub(r'(\[MASK\](\s*)){2,}', r'[MASK]\2', text)

def display_ar_eng(text):
    """Display Arabic and English text in a readable format"""
    display(HTML(f'<div dir="rtl" style="font-size:18px; line-height:1.8; font-family: "Arial", sans-serif;">{text}</div>'))

In [ ]:
train_data['target'] = train_data['target'].apply(merge_consecutive_masks)

display_ar_eng(train_data['source'][0])
display_ar_eng(train_data['target'][0])

In [ ]:
# Save the train data after the updates
# train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

# Fine-tuning

### Loading Data

In [1]:
import pandas as pd
# from google.colab import drive

In [ ]:
# drive.mount('/content/drive')

In [ ]:
# train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv")

In [2]:
# Check the input directory
input_path = "/kaggle/input/arabic-pii/masked_train_data.csv"

# Read the train.csv file
train_data = pd.read_csv(input_path)

# Display the first few rows
train_data.head()


,source,target
0,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...
1,عزيزي جادون، إن بحثك حول دول المراقبة وانتهاكا...,عزيزي [MASK]، إن بحثك حول دول المراقبة وانتهاك...
2,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...
3,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...
4,في 29067 جزر راث، شيد مؤخرا مركز جديد للتربية ...,في [MASK]، شيد مؤخرا مركز جديد للتربية الجنسية...


### Formatting data

In [3]:
from datasets import Dataset
import torch

In [5]:
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Mask the PII words in the given source sentence as "[MASK]".

### Input:
{input}

### Response:
{output}"""

In [6]:
train_data['text'] = train_data.apply(
    lambda row: prompt.format(input=row['source'], output=row['target']),
    axis=1
)

display(train_data[['source', 'target', 'text']].head())

,source,target,text
0,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,Below is an instruction that describes a task....
1,عزيزي جادون، إن بحثك حول دول المراقبة وانتهاكا...,عزيزي [MASK]، إن بحثك حول دول المراقبة وانتهاك...,Below is an instruction that describes a task....
2,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,Below is an instruction that describes a task....
3,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,Below is an instruction that describes a task....
4,في 29067 جزر راث، شيد مؤخرا مركز جديد للتربية ...,في [MASK]، شيد مؤخرا مركز جديد للتربية الجنسية...,Below is an instruction that describes a task....


## Gemma 3 (4B)

### Loading model

In [7]:
# Check versions
!python --version
!nvidia-smi
print("Torch Version:", torch.__version__)

Python 3.11.11
Wed Jun 18 18:25:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------

In [8]:
# ✅ 1. Install specific compatible version
!pip install --upgrade pip
!pip install -qU "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 33.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 98.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 92.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 55.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 86.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 128.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 120.5 MB/s eta 0:00:0000:010:01
   

In [17]:
from unsloth import FastLanguageModel, FastModel
# import os
from kaggle_secrets import UserSecretsClient
import wandb

In [10]:
user_secrets = UserSecretsClient()
wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tuning Gemma-3-4B for Arabic PII Redaction', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: muhammadhelmy (chezlong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [11]:
print(f"Using {torch.cuda.device_count()} GPUs")  # Should print 2 on T4 x2

Using 2 GPUs


In [12]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt",
    max_seq_length = 2048,
    load_in_4bit = True, # True: QLoRA Optimization. False: LoRA Optimization
    dtype = torch.float16,
    full_finetuning = False # False: will use LoRA
)

==((====))==  Unsloth 2025.6.2: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [13]:
# LoRA Optimization
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    
    use_4bit_quants = True, # True: QLoRA
    # target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0.05,
    bias = "none",
    random_state = 101,
    use_gradient_checkpointing = "unsloth" # reduce memory usage by an extra 30% and support extremely long context finetunes
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
# ✅ Enable PyTorch gradient checkpointing (good practice for optimizing the memory usage of deep learning models during training)
# model.gradient_checkpointing_enable()

In [16]:
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(train_data)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        # padding = "max_length",       # Prevent dimension mismatch
        truncation = True,
        max_length = 2048,
        # return_tensors = "pt"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch training
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/376267 [00:00<?, ? examples/s]

### Training the model

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [21]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        output_dir = "outputs",                # where to save checkpoints
        
        save_strategy = "steps",               # save every N steps
        save_steps = 50,                       # adjust as needed
        save_total_limit = 4,                  # keep only last 2 checkpoints
        logging_first_step = True,
        
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        
        warmup_steps = 20,
        max_steps = 300, # adjust this based on your needs
        learning_rate = 2e-5,
        lr_scheduler_type = "cosine",
        
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        dataloader_num_workers = 2,
        logging_steps = 5,
        seed = 101,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


In [22]:
torch.cuda.empty_cache() # Clear GPU Memory Before Training

In [23]:
# os.environ["WANDB_MODE"] = "disabled"
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 376,267 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.375700
5,2.286600
10,2.389700
15,2.319300
20,2.262000
25,2.145700
30,1.971300
35,1.920500
40,1.815800
45,1.730300


In [24]:
trainer_stats

TrainOutput(global_step=300, training_loss=1.4424663337071737, metrics={'train_runtime': 2194.109, 'train_samples_per_second': 2.188, 'train_steps_per_second': 0.137, 'total_flos': 1.8005281164876096e+16, 'train_loss': 1.4424663337071737})

In [25]:
trainer.model.save_pretrained("outputs/gemma_3_4b_pt_model_qlora")
tokenizer.save_pretrained("outputs/gemma_3_4b_pt_model_qlora")

['outputs/gemma_3_4b_pt_model_qlora/processor_config.json']

# Next

# Gemi